<a href="https://colab.research.google.com/github/Vimala-Suram/Agents/blob/main/Assign_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This notebook implements a research agent that uses LLMs to:
1. Break down a topic into subtopics
2. Expand queries with relevant terms
3. Search the web for information
4. Generate and improve summaries
5. Critique and reflect on the research

It uses Together AI's API to access various LLM models, with safeguards
to minimize API costs through caching and mock modes for development.
"""

# ## Setup and Imports

import os
import json
import requests
import time
from typing import List, Dict, Any, Optional, Callable, Tuple
from functools import lru_cache
import hashlib
from google.colab import userdata


# !pip install openai requests python-dotenv

# Import environment variables from .env file (optional but recommended)
try:
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from .env file
    print("Environment variables loaded from .env file")
except ImportError:
    print("python-dotenv not installed, skipping .env file loading")

# Configure the client for Together AI using the v1.0+ OpenAI API format
from openai import OpenAI

# For Colab: Use secrets to avoid exposing API keys
# Uncomment the following lines if using Google Colab
# from google.colab import userdata
# api_key = userdata.get('TOGETHER_API_KEY')

# For local development: Use environment variables
api_key = os.environ.get("TOGETHER_API_KEY")
if not api_key:
    api_key = userdata.get('TOGETHER_API_KEY')
    os.environ["TOGETHER_API_KEY"] = api_key

# Create the OpenAI client with Together AI base URL
client = OpenAI(
    api_key=api_key,
    base_url="https://api.together.xyz/v1"
)

# Set default model (you can change this based on your preference)
# Note: Smaller models are cheaper but may provide lower quality results
DEFAULT_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"  # Good balance of performance and cost
# Other options:
# - "togethercomputer/llama-2-7b-chat" - Lighter model, faster responses
# - "meta-llama/Meta-Llama-3-8B-Instruct" - Another good option
# - "meta-llama/Llama-2-70b-chat-hf" - Larger but more expensive

# Flag to use mock search during development to avoid API costs
USE_MOCK_SEARCH = True  # Set to False when ready to use real search API

print(f"Using model: {DEFAULT_MODEL}")
print(f"Mock search mode: {'Enabled for development purposes only' if USE_MOCK_SEARCH else 'Disabled - will use real search API'}")

# ## Tool 1: Topic Breakdown Tool

def topic_breakdown_tool(topic: str, num_subtopics: int = 3, model: str = DEFAULT_MODEL) -> List[str]:
    """
    Breaks down a broad research topic into smaller, more focused subtopics.

    Args:
        topic: The main research topic
        num_subtopics: Number of subtopics to generate
        model: LLM model to use

    Returns:
        List of subtopics
    """
    # Create a prompt for the LLM
    system_prompt = "You are a research assistant that helps break down topics into focused subtopics."

    user_prompt = f"""
    I need to research the topic: "{topic}"

    Please break this broad topic down into {num_subtopics} specific, focused subtopics that would be useful for research.
    For each subtopic, provide a clear, concise phrase that could be used as a search query.

    Return ONLY the list of subtopics, one per line, without any additional text, numbering, or explanations.
    """

    # Call the LLM using the new API format
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=300,
        temperature=0.3  # Lower temperature for more focused results
    )

    # Extract and clean the subtopics
    subtopics_text = response.choices[0].message.content.strip()
    subtopics = [line.strip() for line in subtopics_text.split('\n') if line.strip()]

    # Ensure we have the requested number of subtopics
    if len(subtopics) < num_subtopics:
        # If we don't have enough, make another call to get more
        print(f"Only got {len(subtopics)} subtopics, requesting more...")

        more_subtopics_prompt = f"""
        I need additional subtopics related to: "{topic}"

        I already have these subtopics:
        {subtopics_text}

        Please provide {num_subtopics - len(subtopics)} more specific, different subtopics.
        Return ONLY the list of additional subtopics, one per line.
        """

        more_response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": more_subtopics_prompt}
            ],
            max_tokens=300,
            temperature=0.4  # Slightly higher temperature for more diverse results
        )

        more_subtopics = [line.strip() for line in more_response.choices[0].message.content.strip().split('\n') if line.strip()]
        subtopics.extend(more_subtopics)

    return subtopics[:num_subtopics]  # Return exactly the requested number

# ## Tool 2: Query Expansion Tool

def query_expansion_tool(query: str, num_expansions: int = 3, model: str = DEFAULT_MODEL) -> List[str]:
    """
    Expands a query with related keywords, synonyms, and phrases.

    Args:
        query: The original search query
        num_expansions: Number of expanded queries to generate
        model: LLM model to use

    Returns:
        List of expanded queries
    """
    system_prompt = "You are a search query optimization specialist."

    user_prompt = f"""
    Original search query: "{query}"

    Please generate {num_expansions} expanded versions of this query by:
    1. Adding relevant keywords
    2. Using synonyms for key terms
    3. Rephrasing to capture the same concept in different ways

    Return ONLY the expanded queries, one per line, without any numbering or explanations.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=300,
        temperature=0.4
    )

    expanded_queries_text = response.choices[0].message.content.strip()
    expanded_queries = [line.strip() for line in expanded_queries_text.split('\n') if line.strip()]

    return expanded_queries[:num_expansions]  # Ensure we only get the requested number

# ## Tool 3: Search Tool

class MockSearchTool:
    """Mock search tool for development without using real API costs"""

    def __init__(self):
        self.cache = {}

    def search(self, query: str, num_results: int = 3) -> List[Dict[str, Any]]:
        """
        Generate mock search results for a query.

        Args:
            query: Search query
            num_results: Number of results to return

        Returns:
            List of mock search results
        """
        # Check if this query is already in the cache
        cache_key = f"{query}_{num_results}"
        if cache_key in self.cache:
            print(f"Using cached mock results for query: {query}")
            return self.cache[cache_key]

        # Generate deterministic but diverse mock results based on query
        query_hash = hashlib.md5(query.encode()).hexdigest()
        results = []

        for i in range(num_results):
            seed = f"{query_hash}_{i}"
            result_hash = hashlib.md5(seed.encode()).hexdigest()

            # Use the hash to generate somewhat diverse mock content
            words = query.split()
            title_words = words + ["research", "analysis", "overview", "studies", "applications", "advances", "technology"]
            snippet_words = words + ["provides", "details", "explains", "examines", "discusses", "presents", "describes", "shows"]

            # Use the hash to seed a simple random choice
            title_idx = [ord(c) % len(title_words) for c in result_hash[:5]]
            snippet_idx = [ord(c) % len(snippet_words) for c in result_hash[5:10]]

            title = f"{' '.join([title_words[idx] for idx in title_idx])} - {query} ({i+1})"
            snippet = f"This {snippet_words[snippet_idx[0]]} {query} and {snippet_words[snippet_idx[1]]} its importance. "
            snippet += f"The article {snippet_words[snippet_idx[2]]} various aspects including {words[ord(result_hash[10]) % len(words)] if words else 'topics'}. "
            snippet += f"Research {snippet_words[snippet_idx[3]]} that {words[ord(result_hash[11]) % len(words)] if words else 'this'} is significant for future developments."

            results.append({
                "title": title.title(),
                "snippet": snippet,
                "url": f"https://example.com/result/{result_hash[:8]}",
                "source": "Mock Search Engine"
            })

        # Cache the results
        self.cache[cache_key] = results
        return results

class SerperSearchTool:
    """
    Search tool using Serper.dev API (Google Search API)
    Documentation: https://serper.dev/docs
    """

    def __init__(self, api_key: str = None):
        self.api_key = api_key or os.environ.get("SERPER_API_KEY")
        if not self.api_key:
            raise ValueError("Serper API key is required")

        self.base_url = "https://google.serper.dev/search"
        self.headers = {
            "X-API-KEY": self.api_key,
            "Content-Type": "application/json"
        }

        # Cache to minimize API calls
        self.cache = {}

    @lru_cache(maxsize=100)
    def search(self, query: str, num_results: int = 5) -> List[Dict[str, Any]]:
        """
        Search the web using Serper.dev API.

        Args:
            query: The search query
            num_results: Number of results to return

        Returns:
            List of search results
        """
        # Check cache first
        cache_key = f"{query}_{num_results}"
        if cache_key in self.cache:
            print(f"Using cached results for query: {query}")
            return self.cache[cache_key]

        payload = {
            "q": query,
            "num": num_results
        }

        try:
            response = requests.post(self.base_url, headers=self.headers, json=payload)
            response.raise_for_status()

            data = response.json()

            # Format the results
            formatted_results = []
            if "organic" in data:
                for item in data["organic"][:num_results]:
                    formatted_results.append({
                        "title": item.get("title", "No title"),
                        "snippet": item.get("snippet", "No snippet available"),
                        "url": item.get("link", ""),
                        "source": f"Serper.dev: {item.get('link', '')}"
                    })

            # Cache the results
            self.cache[cache_key] = formatted_results
            return formatted_results

        except Exception as e:
            print(f"Error in search_tool: {e}")
            return [{
                "title": "Error",
                "snippet": f"An error occurred: {str(e)}",
                "url": "",
                "source": "Error"
            }]

class BraveSearchTool:
    """
    Search tool using Brave Search API
    Documentation: https://brave.com/search/api/
    """

    def __init__(self, api_key: str = None):
        self.api_key = api_key or os.environ.get("BRAVE_API_KEY")
        if not self.api_key:
            raise ValueError("Brave Search API key is required")

        self.base_url = "https://api.search.brave.com/res/v1/web/search"
        self.headers = {
            "Accept": "application/json",
            "Accept-Encoding": "gzip",
            "X-Subscription-Token": self.api_key
        }

        # Cache to minimize API calls
        self.cache = {}

    @lru_cache(maxsize=100)
    def search(self, query: str, num_results: int = 5, country: str = "US") -> List[Dict[str, Any]]:
        """
        Search the web using Brave Search API.

        Args:
            query: The search query
            num_results: Number of results to return
            country: Country code for localized results

        Returns:
            List of search results
        """
        # Check cache first
        cache_key = f"{query}_{num_results}_{country}"
        if cache_key in self.cache:
            print(f"Using cached results for query: {query}")
            return self.cache[cache_key]

        params = {
            "q": query,
            "count": min(num_results, 20),  # API limit is 20
            "country": country,
            "search_lang": "en"
        }

        try:
            response = requests.get(self.base_url, headers=self.headers, params=params)
            response.raise_for_status()

            data = response.json()

            # Format the results
            formatted_results = []
            if "web" in data and "results" in data["web"]:
                for result in data["web"]["results"]:
                    formatted_results.append({
                        "title": result.get("title", "No title"),
                        "snippet": result.get("description", "No description available"),
                        "url": result.get("url", ""),
                        "source": f"Brave Search: {result.get('url', '')}"
                    })

            # Cache the results
            self.cache[cache_key] = formatted_results
            return formatted_results

        except Exception as e:
            print(f"Error in search_tool: {e}")
            return [{
                "title": "Error",
                "snippet": f"An error occurred: {str(e)}",
                "url": "",
                "source": "Error"
            }]

# Create the appropriate search tool based on configuration
def get_search_tool():
    """
    Factory function to create the appropriate search tool
    based on configuration and available API keys.
    """
    if USE_MOCK_SEARCH:
        print("Using mock search tool")
        return MockSearchTool()

    # Try to use Brave Search if API key is available
    brave_api_key = os.environ.get("BRAVE_API_KEY")
    if brave_api_key:
        print("Using Brave Search API")
        return BraveSearchTool(api_key=brave_api_key)

    # Try to use Serper if API key is available
    serper_api_key = os.environ.get("SERPER_API_KEY")
    if serper_api_key:
        print("Using Serper.dev API")
        return SerperSearchTool(api_key=serper_api_key)

    # Fall back to mock search if no API keys are available
    print("No search API keys found, falling back to mock search")
    return MockSearchTool()

# Create the search tool
search_tool_instance = get_search_tool()

def search_tool(query: str, num_results: int = 3) -> List[Dict[str, Any]]:
    """
    Wrapper function to use the appropriate search tool.

    Args:
        query: Search query
        num_results: Number of results to return

    Returns:
        List of search results
    """
    return search_tool_instance.search(query, num_results)

# ## Tool 4: Critique Tool

def critique_tool(summary: str, topic: str, model: str = DEFAULT_MODEL) -> Dict[str, Any]:
    """
    Critiques a research summary and suggests improvements.

    Args:
        summary: The summary to critique
        topic: The original research topic
        model: LLM model to use

    Returns:
        Dictionary with critique and suggestions
    """
    system_prompt = "You are a research critic who provides constructive feedback on summaries."

    user_prompt = f"""
    Research Topic: "{topic}"

    Summary to Review:
    "{summary}"

    Please critique this research summary and provide feedback on:
    1. Accuracy and relevance to the topic
    2. Completeness - are important aspects missing?
    3. Clarity and organization
    4. Potential biases or limitations
    5. Specific suggestions for improvement
    6. Additional subtopics that should be explored

    Format your response as a JSON object with these keys:
    - strengths: list of strengths (2-3 items)
    - weaknesses: list of weaknesses (2-3 items)
    - suggestions: list of specific suggestions for improvement (2-3 items)
    - additional_topics: list of additional subtopics to explore (1-2 items)

    Ensure your response is valid JSON.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=500,
        temperature=0.2
    )

    critique_text = response.choices[0].message.content.strip()

    # Parsing the JSON response
    try:
        critique = json.loads(critique_text)
    except json.JSONDecodeError:
        # If parsing fails creating a simple structure with the raw text
        print("Warning: Failed to parse critique as JSON. Using raw text instead.")
        critique = {
            "strengths": ["Summary provides relevant information"],
            "weaknesses": ["Summary could be improved"],
            "suggestions": ["Consider expanding on key points"],
            "additional_topics": ["Related subtopics"],
            "raw_critique": critique_text
        }

    return critique

# ## Tool 5: Summarizer Tool

def summarizer_tool(content: List[Dict[str, str]], topic: str, model: str = DEFAULT_MODEL) -> str:
    """
    Summarizes content from search results.

    Args:
        content: List of dictionaries with search results
        topic: The original research topic
        model: LLM model to use

    Returns:
        A concise summary paragraph
    """
    # Format the search results into a readable text
    formatted_content = "\n\n".join([
        f"Title: {item['title']}\nSnippet: {item['snippet']}\nURL: {item['url']}"
        for item in content
    ])

    system_prompt = "You are a research assistant that creates concise, accurate summaries."

    user_prompt = f"""
    Research Topic: "{topic}"

    Below are search results about this topic:

    {formatted_content}

    Please synthesize this information into a concise, informative summary paragraph about the topic.
    Focus on key findings, common themes, and important details.
    Keep your summary to approximately 4-6 sentences.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=400,
        temperature=0.4
    )

    summary = response.choices[0].message.content.strip()
    return summary

# ## Research Agent Implementation

class ResearchAgent:
    """
    An agent that uses LLMs and tools to research topics and generate summaries.
    """

    def __init__(self, model: str = DEFAULT_MODEL):
        """
        Initialize the research agent with its tools and model.

        Args:
            model: LLM model to use
        """
        self.model = model
        print(f"Initializing ResearchAgent with model: {model}")

        # Initialize the state
        self.state = {
            "main_topic": "",
            "subtopics": [],
            "expanded_queries": {},
            "search_results": {},
            "current_summary": "",
            "critique": {},
            "final_summary": "",
            "steps_taken": []
        }

    def _log_step(self, step_name: str, input_params: Dict[str, Any], output: Any):
        """Log a step in the research process"""
        self.state["steps_taken"].append({
            "step": step_name,
            "timestamp": time.time(),
            "input": input_params,
            "output_summary": str(output)[:100] + "..." if isinstance(output, str) and len(str(output)) > 100 else str(output)
        })

        print(f"\n--- Step: {step_name} ---")
        print(f"Input: {input_params}")
        if isinstance(output, str):
            print(f"Output: {output[:100]}..." if len(output) > 100 else output)
        elif isinstance(output, list):
            print(f"Output: {len(output)} items")
            for i, item in enumerate(output[:3]):
                print(f"  {i+1}. {str(item)[:50]}...")
            if len(output) > 3:
                print(f"  ... and {len(output) - 3} more")
        elif isinstance(output, dict):
            print(f"Output: Dictionary with keys {list(output.keys())}")
        else:
            print(f"Output: {type(output)}")

    def research(self, topic: str, num_subtopics: int = 3, verbose: bool = True) -> Dict[str, Any]:
        """
        Perform research on a topic and generate a summary.

        Args:
            topic: The research topic
            num_subtopics: Number of subtopics to explore
            verbose: Whether to print progress information

        Returns:
            Dictionary with research results
        """
        # Initialize state for this research session
        self.state = {
            "main_topic": topic,
            "subtopics": [],
            "expanded_queries": {},
            "search_results": {},
            "current_summary": "",
            "critique": {},
            "final_summary": "",
            "steps_taken": []
        }

        if verbose:
            print(f"Starting research on topic: {topic}")

        # Step 1: Break down the topic into subtopics
        subtopics = topic_breakdown_tool(topic, num_subtopics, self.model)
        self.state["subtopics"] = subtopics
        self._log_step("topic_breakdown", {"topic": topic, "num_subtopics": num_subtopics}, subtopics)

        # Step 2: Expand queries for the main topic and subtopics
        # First expand the main topic
        expanded_main_queries = query_expansion_tool(topic, 3, self.model)
        self.state["expanded_queries"][topic] = expanded_main_queries
        self._log_step("query_expansion_main", {"query": topic}, expanded_main_queries)

        # Then expand each subtopic
        for subtopic in subtopics:
            expanded_queries = query_expansion_tool(subtopic, 2, self.model)
            self.state["expanded_queries"][subtopic] = expanded_queries
            self._log_step("query_expansion_subtopic", {"query": subtopic}, expanded_queries)

        # Step 3: Search for information
        # First, search the main topic using the first expanded query
        main_query = self.state["expanded_queries"][topic][0] if self.state["expanded_queries"][topic] else topic
        main_results = search_tool(main_query, 3)
        self.state["search_results"][topic] = main_results
        self._log_step("search_main_topic", {"query": main_query, "num_results": 3}, main_results)

        # Then search for each subtopic
        all_search_results = main_results.copy()
        for subtopic in subtopics:
            # Use the first expanded query if available
            subtopic_query = self.state["expanded_queries"][subtopic][0] if self.state["expanded_queries"][subtopic] else subtopic
            results = search_tool(subtopic_query, 2)
            self.state["search_results"][subtopic] = results
            all_search_results.extend(results)
            self._log_step("search_subtopic", {"query": subtopic_query, "subtopic": subtopic, "num_results": 2}, results)

        # Step 4: Generate an initial summary
        initial_summary = summarizer_tool(all_search_results, topic, self.model)
        self.state["current_summary"] = initial_summary
        self._log_step("generate_summary", {"topic": topic, "num_results": len(all_search_results)}, initial_summary)

        # Step 5: Critique the summary
        critique = critique_tool(initial_summary, topic, self.model)
        self.state["critique"] = critique
        self._log_step("critique_summary", {"summary": initial_summary, "topic": topic}, critique)

        # Step 6: Improve the summary based on the critique
        # Create a prompt for improving the summary
        system_prompt = "You are a research assistant that improves summaries based on feedback."

        improvement_prompt = f"""
        Original Research Topic: "{topic}"

        Original Summary:
        "{initial_summary}"

        Critique:
        - Strengths: {', '.join(critique['strengths'])}
        - Weaknesses: {', '.join(critique['weaknesses'])}
        - Suggestions: {', '.join(critique['suggestions'])}

        Please rewrite the summary to address these weaknesses and incorporate the suggestions.
        Create an improved, comprehensive yet concise paragraph (4-6 sentences).
        """

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": improvement_prompt}
            ],
            max_tokens=400,
            temperature=0.4
        )

        final_summary = response.choices[0].message.content.strip()
        self.state["final_summary"] = final_summary
        self._log_step("improve_summary", {"original_summary": initial_summary, "critique": critique, "topic": topic}, final_summary)

        if verbose:
            print("\n" + "="*50)
            print("RESEARCH COMPLETE")
            print("="*50)
            print(f"Topic: {topic}")
            print("\nSubtopics explored:")
            for subtopic in subtopics:
                print(f"- {subtopic}")
            print("\nFinal Summary:")
            print(final_summary)
            print("="*50)

        # Return the research results
        return {
            "topic": topic,
            "subtopics": subtopics,
            "search_results": self.state["search_results"],
            "initial_summary": initial_summary,
            "critique": critique,
            "final_summary": final_summary,
            "steps": self.state["steps_taken"]
        }

# Interactive function to run the agent
def run_interactive():
    """Run the research agent interactively"""
    print("\n=== Research Agent ===\n")
    topic = input("Enter a research topic: ")

    num_subtopics = 3
    try:
        num_subtopics_input = input("Number of subtopics to explore (default: 3): ")
        if num_subtopics_input.strip():
            num_subtopics = int(num_subtopics_input)
    except ValueError:
        print("Invalid input. Using default value of 3 subtopics.")

    model = DEFAULT_MODEL
    model_input = input(f"LLM Model to use (default: {DEFAULT_MODEL}): ")
    if model_input.strip():
        model = model_input

    print(f"\nResearching '{topic}' with {num_subtopics} subtopics using {model}...\n")

    agent = ResearchAgent(model=model)
    results = agent.research(topic, num_subtopics=num_subtopics, verbose=True)

    # Ask if the user wants to see detailed results
    show_details = input("\nShow detailed research results? (y/n): ").lower() == 'y'

    if show_details:
        print("\n" + "="*50)
        print("DETAILED RESEARCH RESULTS")
        print("="*50)

        print("SUBTOPICS:")
        for i, subtopic in enumerate(results["subtopics"], 1):
            print(f"{i}. {subtopic}")

            # Show expanded queries for this subtopic
            if subtopic in agent.state["expanded_queries"]:
                print("   Expanded queries:")
                for j, query in enumerate(agent.state["expanded_queries"][subtopic], 1):
                    print(f"   {j}. {query}")

            # Show search results for this subtopic
            if subtopic in results["search_results"]:
                print("   Search results:")
                for j, result in enumerate(results["search_results"][subtopic], 1):
                    print(f"   {j}. {result['title']}")
                    print(f"      {result['snippet'][:100]}...")

            print()

        print("\nINITIAL SUMMARY:")
        print(results["initial_summary"])

        print("\nCRITIQUE:")
        print("Strengths:")
        for strength in results["critique"]["strengths"]:
            print(f"- {strength}")
        print("Weaknesses:")
        for weakness in results["critique"]["weaknesses"]:
            print(f"- {weakness}")
        print("Suggestions:")
        for suggestion in results["critique"]["suggestions"]:
            print(f"- {suggestion}")
        if "additional_topics" in results["critique"]:
            print("Additional topics to explore:")
            for topic in results["critique"]["additional_topics"]:
                print(f"- {topic}")

        print("\nFINAL SUMMARY:")
        print(results["final_summary"])

        print("="*50)

    return results


if __name__ == "__main__":
    run_interactive()
else:
    print("Import complete. Use ResearchAgent class or run_interactive() function to start researching.")

# %% [markdown]
# ## Testing the Research Agent
#
# Let's test the agent with a sample topic:

# %%
# Uncomment to run a test
# agent = ResearchAgent()
# test_results = agent.research("Quantum computing applications in healthcare", verbose=True)
# print(test_results["final_summary"])

Environment variables loaded from .env file
Using model: mistralai/Mixtral-8x7B-Instruct-v0.1
Mock search mode: Enabled for development purposes only
Using mock search tool

=== Research Agent ===

Enter a research topic: Learning deep features for discriminative localization
Number of subtopics to explore (default: 3): 2
LLM Model to use (default: mistralai/Mixtral-8x7B-Instruct-v0.1): mistralai/Mixtral-8x7B-Instruct-v0.1

Researching 'Learning deep features for discriminative localization' with 2 subtopics using mistralai/Mixtral-8x7B-Instruct-v0.1...

Initializing ResearchAgent with model: mistralai/Mixtral-8x7B-Instruct-v0.1
Starting research on topic: Learning deep features for discriminative localization

--- Step: topic_breakdown ---
Input: {'topic': 'Learning deep features for discriminative localization', 'num_subtopics': 2}
Output: 2 items
  1. "Deep learning methods for feature extraction in d...
  2. "Comparison of deep feature-based approaches for d...

--- Step: query_exp

# New Section